In [1]:
import numpy as np
import cvxpy as cp
import pandas as pd
import matplotlib.pyplot as plt

/Users/anandsranjan/opt/anaconda3/envs/nfs_env/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/anandsranjan/opt/anaconda3/envs/nfs_env/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/anandsranjan/opt/anaconda3/envs/nfs_env/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [25]:
def gen_synthetic_multinomial(n0,n1,q,r,j):
    # n0,n1 is the number of samples from class 0, 1 resp.
    # q,r is are the theta^+/-
    
    X_train = []
    y_train = []
    
    X_train.extend(np.random.multinomial(j, q, size=n0))
    y_train.extend([1]*n0)
    
    X_train.extend(np.random.multinomial(j, r, size=n1))
    y_train.extend([-1]*n1)
    
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    
    return X_train, y_train

def gen_synthetic_normal(n,d,sigma):
    
    X = np.random.normal(0, 1, n*d).reshape(n, d)
    w = np.random.normal(0, 1, d)
    y = X@w + np.random.normal(0, sigma, n)
    
    return X, np.array([y]).T, np.array([w]).T

## Create dataset

In [89]:
n = 100 # number of data points
m = 10 # number of features
sigma = 5
lamb = 10
rho = 5
X, y, w = gen_synthetic_normal(n,m,sigma)

In [90]:
print("Shape of X: ", X.shape)
print("Shape of y: ", y.shape)
print("Shape of w: ", w.shape)

Shape of X:  (100, 10)
Shape of y:  (100, 1)
Shape of w:  (10, 1)


## Nominal Problem

In [94]:
def solve_nominal(X, y, lamb=10):
    w = cp.Variable((m,1))
    nominal_obj = cp.Minimize(cp.square(cp.norm(X@w - y, 2)) + lamb*cp.square(cp.norm(w,2)))
    constraints = []
    prob = cp.Problem(nominal_obj, constraints)
    prob.solve()
    return prob.value, w.value

In [95]:
loss, w_hat = solve_nominal(X,y,lamb=10)

In [96]:
print("Loss: ", loss)

Loss:  2100.2663463521494


## Robust Problem

In [97]:
def solve_robust(X,y, rho, lambda_):
    # data, labels = X,y
    n,d = X.shape
    w_rob = cp.Variable((d,1))

    abs_resids = cp.abs(X@w_rob - y)
    robper = rho*cp.norm(w_rob, p=2)*np.ones((n,1))
    ridge_pen = lambda_*cp.norm(w_rob, p=2)**2
    obj = cp.norm(abs_resids + robper, p=2)**2 + ridge_pen

    prob = cp.Problem(cp.Minimize(obj))
    prob.solve()
    return prob.value, w_rob.value

In [98]:
loss, w_hat = solve_robust(X,y,rho=rho,lambda_=lamb)

In [99]:
print("Loss: ", loss)

Loss:  3051.7010391679364


## Poisoned Problem

In [115]:
def solve_poisoned(X_nominal, y, lamb=10, rho=10):
    X = cp.Variable((n,m))
    M = cp.Variable((m+1,m+1), PSD=True)
    N = cp.Variable((m+n,m+n), PSD=True)
    t = cp.Variable(1)
    U = cp.Variable((m,m), PSD=True)
    poisoned_obj = cp.Minimize(t - cp.square(cp.norm(y)) + cp.trace(U))
    constraints = [
        M[0:m,0:m] == U,
        M[m,m] == t,
        M[m:m+1,0:m] == y.T@X,
        M[0:m,m:m+1] == X.T@y,
        N[0:m,0:m] == U,
        N[m:,m:] == np.eye(n),
        N[m:,0:m] == X,
        N[0:m,m:] == X.T,
        
    ]
    for i in range(n):
        constraints.append(cp.norm(X[i]-X_nominal[i],2) <= rho)
    prob = cp.Problem(poisoned_obj, constraints)
    prob.solve()
    return -1*prob.value, X.value, U.value

In [116]:
print("Shape of X: ", X.shape)
print("Shape of y: ", y.shape)
print("Shape of w: ", w.shape)

Shape of X:  (100, 10)
Shape of y:  (100, 1)
Shape of w:  (10, 1)


In [117]:
print((y.T@X).shape)
print((np.ones((m+1,m+1))[m:m+1,0:m]).shape)

(1, 10)
(1, 10)


In [118]:
loss, X_hat, U_hat = solve_poisoned(X,y,lamb=lamb,rho=rho)

In [119]:
print(loss)

3051.7011503086014


In [120]:
U_hat

array([[ 1.42827661e-10,  1.67923291e-11,  1.27819559e-11,
        -1.18503068e-11, -1.34602954e-11, -1.25208611e-11,
         2.59324232e-12, -1.40812724e-11,  1.35851672e-11,
         5.05746511e-12],
       [ 1.67923291e-11,  1.63700790e-10,  3.03679626e-11,
         2.77873337e-11,  3.08350066e-11,  2.21666435e-11,
         1.59945695e-11,  2.93182999e-11,  2.00019537e-11,
         2.46214093e-11],
       [ 1.27819559e-11,  3.03679626e-11,  2.04239990e-10,
         1.56658162e-11,  2.45900608e-11,  9.85814326e-12,
         1.11764152e-11, -2.77804562e-11,  1.70344372e-11,
        -1.33027359e-11],
       [-1.18503068e-11,  2.77873337e-11,  1.56658162e-11,
         1.56675380e-10,  1.07811666e-11,  1.59225176e-12,
        -3.45139635e-12, -5.82010916e-11,  4.58109719e-12,
        -6.92146903e-12],
       [-1.34602954e-11,  3.08350066e-11,  2.45900608e-11,
         1.07811666e-11,  1.60977928e-10,  2.64640080e-12,
         2.27474068e-12, -1.06763968e-11, -6.71247752e-12,
        -1.

In [121]:
X

array([[ 2.32676573e+00,  1.20826970e+00,  4.33341015e-01,
         4.02716416e-01, -1.28622691e-01, -1.19074618e+00,
         5.65025067e-03, -7.89265483e-01, -7.40669238e-01,
         1.48770118e-01],
       [ 8.75321872e-01,  9.06041214e-01,  2.81529512e-01,
        -8.51715016e-01, -2.13792841e-01, -1.06063711e+00,
         7.01357320e-01,  2.11756890e-01, -3.86045095e-01,
         2.77325763e-01],
       [-5.58870398e-01, -1.27085408e-01, -1.46429243e+00,
         1.87051496e+00,  7.26635533e-02, -3.67345109e-01,
         1.19543651e+00, -1.95687694e+00,  5.10022469e-01,
         1.02208860e+00],
       [-2.37596385e-01, -1.26734023e+00, -1.90156527e+00,
        -2.79941581e+00, -1.62420617e+00, -1.17666562e+00,
         8.17904823e-01, -7.93840100e-01,  1.62446670e+00,
        -1.08646735e+00],
       [ 3.29633616e-01, -6.96011242e-01,  8.37862925e-01,
        -1.15836966e+00, -3.12479123e-01,  8.68282111e-01,
        -9.41515739e-01, -1.67048408e-01,  4.40490823e-01,
        -5.